<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B210516%5DMake_critical_Ising_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    base='Google Drive'

In [80]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm, tnrange
import pickle as pkl
import random

In [18]:
def Ising_model(config, T, n0, n_delta, n_sample):
    #n0는 equlibrium에 도달하는데 걸리는 시간
    #n_delta는 correlation이 줄어드는데 걸리는 시간
    #n_sample은 sampling하고 싶어은 샘플의 숫자
    L = len(config)
    samples=[]
    equil,count=0,0
    #alpha
    sampled=0 #샘플링된 샘플들의 숫자를 셉니다. 
    while sampled < n_sample: #원하는 갯수만큼 sampling을 해야 하므로 이 부분은 while문이 와야 합니다.
        if equil < n0: #equilibrium에 도달할 때까지는 샘플을 수집해서는 안됩니다. 
            for i in range(L):
                s = config[i]
                neigh=config[(i+1)%L]+config[(i-1)%L] 
                DE = 2 * s * neigh 

                if DE < 0:
                    s *= -1 
                elif np.random.rand() < np.exp(- DE*1/T):
                    s *= -1 
                config[i] = s
            equil += 1;

        else: #이제는 sample을 수집해도 됩니다.
            x = np.random.randint(L)
            s = config[x]  
            neigh=config[(x+1)%L]+config[(x-1)%L] 
            DE = 2 * s * neigh 

            #accept or reject 
            if DE < 0:
                s *= -1 

            elif np.random.rand() < np.exp(- DE*1/T):
                s *= -1 

            config[x] = s
            count += 1;
            equil+=1; #이건 굳이 없어도 됩니다. 
            if count == n_delta: #correlation이 큰 샘플은 버려야 합니다. 따라서 n_delta라는 step차이를 두고 sampling을 해야 합니다.
                count = 0;
                a=config.copy()
                samples.append(np.float32(np.reshape(a,(1,L))+1)/2)
                #beta
                sampled+=1; 
                
    return samples

In [75]:
L=9
config1 = - np.ones((L))




n0 = 2000
n_delta = 50
n_sample=51200
mcmax = n_sample+n0


In [ ]:
for T in [1.9,3.6,16]:
    list0=[]
    for i in tnrange(10):
        lat = Ising_model(config1, T, n0, n_delta, n_sample)
        random.shuffle(lat)
        random.shuffle(lat)
        list0.append(lat)
    with open('{base}/loss_IG/9*1/9*1_full_T={T}.pkl'.format(base=base, T=T), 'wb') as f:
        pkl.dump(list0, f)